**Окатьева А.Д. ДЗ №3**
* Выдвижение гипотезы
* Построение пайплайна для проверки гипотезы

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import os
from datetime import date, timedelta

raw_df_time = pd.read_csv('../input/coronavirusdataset/Time.csv')
raw_df_time.head

In [ ]:
# Преобразование датасета Time.csv
def df_time_preprocessing(raw_df_time):
    date = pd.to_datetime(raw_df_time['date']) 
    date_shifted = date - timedelta(14) 
    date_delta = (date_shifted - date_shifted.min()) / np.timedelta64(1,'D') 
    percentage_positive = raw_df_time['confirmed'] / raw_df_time['test']  
    return pd.DataFrame({"date": date_shifted,                           
                         "delta": date_delta,                            
                         "test": raw_df_time["test"],                   
                         "percentage_positive": percentage_positive})

def filter_by_positive_rate(df_time):
    return df_time[df_time['percentage_positive'] < 0.05]

In [ ]:
df_time = filter_by_positive_rate(df_time_preprocessing(raw_df_time))
df_time.head()

In [ ]:
# Посмотрим каким образом изменяется процент положительных тестов с даты начала исследования
sns.regplot(x="delta", y="percentage_positive", data=df_time).set_title("percentage confirmed");

In [ ]:
# Посмотрим на изменение общего количества тестов
sns.regplot(x="delta", y="test", data=df_time).set_title("number of tests");

In [ ]:
sns.displot(data=df_time["percentage_positive"], bins=25, kde=True);
print(f"Normal test p-value: {stats.normaltest(df_time['percentage_positive']).pvalue}")

In [ ]:
sns.displot(data=np.sqrt(df_time['percentage_positive']), bins=25, kde=True)
print(f"Normal test p-value: {stats.normaltest(np.sqrt(df_time['percentage_positive'])).pvalue}")

In [ ]:
# После того как подтвердили что sqrt подошло, сделаем отдельную функцию нормализации для данных
def normalize_sqrt(df, col_name):
    new_df = df.copy()
    new_df[col_name] = np.sqrt(new_df[col_name])
    return new_df

df_time_norm = normalize_sqrt(df_time, "percentage_positive")

In [ ]:
#загрузим данные о поисках пользователей по теме заболеваний
raw_df_search = pd.read_csv('../input/coronavirusdataset/SearchTrend.csv')  
raw_df_search.head()

In [ ]:
# с помощью функции преобразуем данные столбца "date" в формат datetime64[ns] для последующего объединения с df_time_norm

def datetype(df):
    newtype_df = df.copy()
    newtype_df.date = df.date.astype('datetime64[ns]')
    return newtype_df

df_search = datetype(raw_df_search)
df_search.info() # удостоверимся в преобразовании

In [ ]:
df_total = raw_df_search.merge(df_time_norm, on="date") # объединим 2 датасета df_time_norm и raw_df_search по дате
df_total.head()

In [ ]:
Теперь сделаем то же самое в виде пайплайнов

In [ ]:
from sklearn.preprocessing import FunctionTransformer as FT
from sklearn.pipeline import Pipeline

pipe_time = Pipeline([("df_time_preprocessing",FT(df_time_preprocessing)),
                      ("filter_by_positive_rate", FT(filter_by_positive_rate)),
                      ("normalize_sqrt", FT(normalize_sqrt,kw_args={"col_name": "percentage_positive"}))]) 
pipe_search = Pipeline([("datetype", FT(datetype))])

raw_df_time = pd.read_csv('../input/coronavirusdataset/Time.csv')
raw_df_search = pd.read_csv('../input/coronavirusdataset/SearchTrend.csv')  

df_total = pipe_search.transform(raw_df_search).merge(pipe_time.transform(raw_df_time), on="date")

df_total.head()

С помощью пайплайнов получили тот же результат. Далее проанализируем полученный датасет.

In [ ]:
sns.scatterplot(x="date", y="cold", size="percentage_positive", hue="percentage_positive", 
                data=df_total, sizes=(20, 200)).set_title("cold vs percentage positive");

На графике явно заметен пик активности поисковых запросов, содержащих слово "cold" во время всплеска заболеваемости коронавирусом. Можно сделать вывод о том, что в период эпидемии многие чувствовали недомогание, но надеялись на обычную простуду. Однако частота запросов снижается раньше частоты выявления заболеваний, что говорит о действии программы информирования населения.

In [ ]:
sns.scatterplot(x="date", y="flu", size="percentage_positive", hue="percentage_positive", 
                data=df_total, sizes=(20, 200)).set_title("flu vs percentage positive");

Обилие запросов "flu" в начале эпидемии говорит о том, что население не было информировано с новой болезнью, скорее всего списывали симптомы коронавируса на грипп. По мере распространения информации число запросов снижается.

In [ ]:
sns.scatterplot(x="date", y="pneumonia", size="percentage_positive", hue="percentage_positive", 
                data=df_total, sizes=(20, 200)).set_title("pneumonia vs percentage positive");

Пневмония, как одно из возможных осложнений после коронавируса интересовало население только в начале эпидемии, далее пики частоты запросов совпадают с пиками заболеваемости.

In [ ]:
sns.scatterplot(x="date", y="coronavirus", size="percentage_positive", hue="percentage_positive", 
                data=df_total, sizes=(20, 200)).set_title("coronavirus vs percentage positive");

Сравнивая с другими графиками, можно сделать очевидный вывод - "коронавирус" являлся самым популярным запросом, на втором месте "простуда". 

**Таким образом, проанализировав данные, можно сделать вывод о распространении информации об эпидемии и отношении к этому граждан. Многие надеялись на сезонную простуду и, соотвественно, не принимали необходимые меры безопасности, что могло негативно отразиться на скорости распространения заболевания.**